In [29]:
# Find "The shortest route to iterate all subway lines in ShangHai."
# Answers can be found on Zhihu: https://www.zhihu.com/question/629922852/answer/54798222956

import gurobipy as grb

In [185]:
def read_metro_data(file_path):
    from collections import defaultdict
    # 邻接表表示的地铁网络图
    metro_network = defaultdict(dict)
    line_routes = dict()
    station_name_to_idx = dict()
    idx_to_station_name = dict()
    cnt_idx = 1
    # transship = dict()
    with open(file_path, 'r', encoding='utf-8') as file:
        current_line = None
        for line in file:
            line = line.strip()
            if line.startswith("Line"):
                # 读取线路名
                current_line = line
                line_routes[line] = list()
            elif line:
                # 读取站点和距离信息
                station1, station2, distance = line.split(",")
                if station1 not in station_name_to_idx:
                    station_name_to_idx[station1] = cnt_idx
                    idx_to_station_name[cnt_idx] = station1
                    cnt_idx += 1
                if station2 not in station_name_to_idx:
                    station_name_to_idx[station2] = cnt_idx
                    idx_to_station_name[cnt_idx] = station2
                    cnt_idx += 1
                distance = int(distance)  # 转换距离为整数
                metro_network[station1][station2] = distance
                metro_network[station2][station1] = distance
                # metro_network[station1][station2] = 1
                # metro_network[station2][station1] = 1
                line_routes[current_line].append(f"{station1},{station2}")
                

    return metro_network, line_routes, station_name_to_idx, idx_to_station_name

# 示例调用
file_path = "./data/PublicTransit/SH.txt"  # 替换为你的实际文件路径
metro_network, line_routes, station_name_to_idx, idx_to_station_name = read_metro_data(file_path)

# 打印部分结果查看结构
for station, connections in list(metro_network.items())[:5]:
    print(f"Station: {station}, Connections: {connections}")

Station: 莘庄, Connections: {'外环路': 1303, '春申路': 1588}
Station: 外环路, Connections: {'莘庄': 1303, '莲花路': 1459}
Station: 莲花路, Connections: {'外环路': 1459, '锦江乐园': 1633}
Station: 锦江乐园, Connections: {'莲花路': 1633, '上海南站': 2089}
Station: 上海南站, Connections: {'锦江乐园': 2089, '漕宝路': 1661, '石龙路': 1314, '华东理工大学': 1623, '桂林公园': 1828}


In [186]:
ptn = grb.Model("SH_PTN")
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0
x = {}
y = {}
n = {}
N = len(station_name_to_idx.keys())
M = 70
out_flow = dict()

for i in range(1, N):
    x[0, i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_{idx_to_station_name[i]}_Dummy")
    y[i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"y_{idx_to_station_name[i]}")
    n[i] = ptn.addVar(vtype=grb.GRB.INTEGER,obj = 0, lb = 0, name=f"n_{idx_to_station_name[i]}")
    out_flow[i] = []

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = dist, name=f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)
# 针对换乘的情况, 给出带转运的解


# 流出的
for k, v in out_flow.items():

    v_new = v + [0]
    ptn.addConstr(grb.quicksum(x[k, j] for j in v_new) == y[k], name=f"out_flow_{idx_to_station_name[k]}")
    ptn.addConstr(grb.quicksum(x[i, k] for i in v_new) == y[k], name=f"in_flow_{idx_to_station_name[k]}")


ptn.addConstr(grb.quicksum(x[i, 0] for i in range(1, N)) == 1, name=f"Dummy_balance1")
ptn.addConstr(grb.quicksum(x[0, i] for i in range(1, N)) == 1, name=f"Dummy_balance2")

for i in range(1, N):
    ptn.addConstr(n[i] <= M * y[i], name = f"N_{i}")

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1), name = f"n_{idx_1}_{idx_2}")

for k, edges in line_routes.items():
    left_edges = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        left_edges.append([o_idx, d_idx])
        left_edges.append([d_idx, o_idx])
    ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 1, name = f"line_{k}")

# 处理 Line 3 + Line 4 的情况
edges_3, edges_4 = line_routes["Line 3"], line_routes["Line 4"]
new_edges = list(set(edges_3 + edges_4))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_3/4")

edges_2, edges_10 = line_routes["Line 2"], line_routes["Line 10"]
new_edges = list(set(edges_2 + edges_10))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_10/2")


ptn.modelSense = grb.GRB.MINIMIZE
ptn.optimize()
if (ptn.status == grb.GRB.status.OPTIMAL):
    solType = 'IP_Optimal'
    ofv = ptn.getObjective().getValue()
    print(f"BEST VALUE:{ofv}")
    for i, j in x:
        if (x[i, j].x > 0.5):
            print(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")

    gap = 0
    lb = ofv
    ub = ofv
    runtime = ptn.Runtime

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2203 rows, 2586 columns and 9276 nonzeros
Model fingerprint: 0xb9763a16
Variable types: 0 continuous, 2586 integer (2181 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [6e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve time: 0.01s
Presolved: 2203 rows, 2586 columns, 9276 nonzeros
Variable types: 0 continuous, 2586 integer (2181 binary)

Root relaxation: objective 1.541800e+04, 132 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15418.0000    0   56          - 15418.0000      -     -    0s
     0     0 16538.0000    0   60          - 16538.0000 

In [ ]:
# 【所需边数最小的（44条边）】
# 国家会展中心L17,虹桥火车站,2523
# 虹桥火车站,虹桥2号航站楼,696
# 虹桥2号航站楼,淞虹路,5921
# 淞虹路,北新泾,1367
# 北新泾,威宁路,1260
# 威宁路,娄山关路,1661
# 娄山关路,长风公园,1487
# 长风公园,大渡河路,784
# 大渡河路,金沙江路,1676
# 金沙江路,曹杨路,903
# 曹杨路,武宁,1050
# 武宁,武定路,970
# 武定路,静安寺,1030
# 静安寺,常熟路,1091
# 常熟路,陕西南路,939
# 陕西南路,一大会址·新天地,1591
# 一大会址·新天地,老西门,880
# 老西门,陆家浜路,825
# 陆家浜路,小南门,1475
# 小南门,商城路,2398
# 商城路,世纪大道,979
# 世纪大道,源深体育中心,890
# 源深体育中心,民生路,907
# 民生路,杨高中路,928
# 杨高中路,迎春路,633
# 迎春路,龙阳路,2275
# 龙阳路,华夏中路,4355
# 华夏中路,莲溪路,1589
# 莲溪路,御桥,1342
# 御桥,浦三路,3221
# 浦三路,三林东,1613
# 三林东,三林,1126
# 三林,东方体育中心,3516
# 东方体育中心,龙耀路,2164
# 龙耀路,云锦路,934
# 云锦路,龙华,945
# 龙华,龙漕路,1124
# 龙漕路,石龙路,1484
# 石龙路,上海南站,1314
# 上海南站,锦江乐园,2089
# 锦江乐园,莲花路,1633
# 莲花路,外环路,1459
# 外环路,莘庄,1303
# 莘庄,春申路,1588
# 69938 



| 序号 | 线路 | 起点 | 终点| 距离 | 
|:---:| :---: |:----: | :---:| :---: |
| 1 | L17 | 国家会展中心L17 | 虹桥火车站 | 2523 |
| 2 | L2 | 虹桥火车站 | 虹桥2号航站楼 | 696 | 
| 3 | L10 | 虹桥2号航站楼 | 虹桥1号航站楼 | 2026 |
|4 | L10  | 虹桥1号航站楼 | 上海动物园 | 2016 |
| 5 | L10 | 上海动物园 | 龙溪路 | 1244| 
|6 | L10| 龙溪路 | 水城路 | 1303 |
| 7| L10| 水城路 | 伊犁路 | 1167 |
| 8| L10 | 伊犁路 | 宋园路 | 800 |
| 9| L10| 宋园路 | 虹桥路 | 1053 |
| 10 | L3 | 虹桥路 | 宜山路 | 1323 |
| 11| L9 | 宜山路 | 徐家汇 | 1505 |
| 12| L11 | 徐家汇 | 交通大学 | 905 |
| 13| L11 | 交通大学 | 上海图书馆 | 1145 |
| 14| L11 | 上海图书馆 | 陕西南路 | 1618 |
| 15 | L1 | 陕西南路 | 一大会址·黄陂南路 | 1407 |
|16 |  L14 | 一大会址·黄陂南路 | 大世界 | 710 | 
| 17| L14 | 大世界 | 豫园 | 740 |
|8| L14 | 豫园 | 陆家嘴 | 1600 |
|9| L14 | 陆家嘴 | 浦东南路 | 1010 |
|20| L2 | 浦东南路 | 世纪大道 | 1245 |
|21| L6 | 世纪大道 | 源深体育中心 | 890 |
|22| L6| 源深体育中心 | 民生路 | 907 |
|23| L18 | 民生路 | 杨高中路 | 928 |
|24| L18| 杨高中路 | 迎春路 | 633 |
|25| L18 | 迎春路 | 龙阳路 | 2275 |
|26| L16 | 龙阳路 | 华夏中路 | 4355 |
|27| L13 | 华夏中路 | 莲溪路 | 1589 |
|28| L13| 莲溪路 | 陈春路 | 1239 |
|29 |L13 | 陈春路 | 北蔡 | 980 |
|30 | L13| 北蔡 | 下南路 | 1128 |
|31|L13 | 下南路 | 华鹏路 | 1361 |
|32| L13| 华鹏路 | 东明路 | 1464 |
|33 | L13| 东明路 | 成山路 | 1397 |
|34 |L8 | 成山路 | 耀华路 | 908 |
|35| L7 | 耀华路 | 长清路 | 926 |
|36| L7| 长清路 | 后滩 | 1263 |
|37| L7| 后滩 | 龙华中路 | 2247 |
|38| L7| 龙华中路 | 东安路 | 728 |
|39| L4| 东安路 | 上海体育场 | 1220 |
|40 | L4| 上海体育场 | 上海体育馆 | 685 |
|41 |L1 | 上海体育馆 | 漕宝路 | 1534 |
|42 |L12 | 漕宝路 | 桂林公园 | 1284 |
|43 |L15 | 桂林公园 | 上海南站 | 1828 |
|44 |L1 | 上海南站 | 锦江乐园 | 2089 |
|45 |L1 | 锦江乐园 | 莲花路 | 1633 |
|46 |L1 | 莲花路 | 外环路 | 1459 |
|47 |L1 | 外环路 | 莘庄 | 1303 |
|48 |L5 | 莘庄 | 春申路 | 1588 |
